<a href="https://colab.research.google.com/github/chouhandiksha/bigdataproject/blob/colab/notebooks/diksha/analyse_LA_completely_home_mobility_poverty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Spark SQL Documentation:** 
https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

In [ ]:
!pip install ipython-autotime

%load_ext autotime

time: 1.98 ms (started: 2021-04-09 23:53:15 +00:00)


In [ ]:
# Install required dependancies
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 212.3MB 71kB/s 
     |████████████████████████████████| 204kB 32.9MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=464c06265bd32d08ad0c2b1e5a65796d8bbc652c0d9caa146a677fffcc36650b
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 31 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading

In [ ]:
# Import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

time: 94 ms (started: 2021-04-09 23:54:47 +00:00)


In [ ]:
import altair as alt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

time: 257 ms (started: 2021-04-09 23:54:49 +00:00)


In [ ]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

time: 4.76 s (started: 2021-04-09 23:54:51 +00:00)


In [ ]:
# Mount drive with data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
time: 20.6 s (started: 2021-04-09 23:54:58 +00:00)


In [ ]:
# Demographic Data LA
# !ls drive/MyDrive/big-data-project/data/clean-data/la/la.csv

# !ls drive/MyDrive/big-data-project/data/clean-data/la/social/2020/



time: 2.28 ms (started: 2021-04-06 17:22:21 +00:00)


In [ ]:
# Set path to data folder
path = Path('drive/MyDrive/big-data-project/data/clean-data')
city = 'la'

time: 1.55 ms (started: 2021-04-09 23:55:22 +00:00)


In [ ]:
# Read data into RDD
df_soc = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/'social/2020/*.csv'))
df_soc.show()

+---+------------+--------------------+--------------------+------------+---------------------------+--------------------------+------------------------------------------+----------------------------+----------------------+------------------------+--------------------+-------------------------------+-------------------------------+--------------------+-------------------------+--------------------------+----------------------+----------------------------+---------------------------+-----------------------------+
|_c0|         cbg|    date_range_start|      date_range_end|device_count|distance_traveled_from_home|bucketed_distance_traveled|median_dwell_at_bucketed_distance_traveled|completely_home_device_count|median_home_dwell_time|bucketed_home_dwell_time|at_home_by_each_hour|part_time_work_behavior_devices|full_time_work_behavior_devices|    destination_cbgs|delivery_behavior_devices|median_non_home_dwell_time|candidate_device_count|bucketed_away_from_home_time|median_percentage_time_

In [ ]:
# df_soc = df_soc.sort("device_count")
# df_soc.show()

+----+------------+--------------------+--------------------+------------+---------------------------+--------------------------+------------------------------------------+----------------------------+----------------------+------------------------+--------------------+-------------------------------+-------------------------------+--------------------+-------------------------+--------------------------+----------------------+----------------------------+---------------------------+-----------------------------+
| _c0|         cbg|    date_range_start|      date_range_end|device_count|distance_traveled_from_home|bucketed_distance_traveled|median_dwell_at_bucketed_distance_traveled|completely_home_device_count|median_home_dwell_time|bucketed_home_dwell_time|at_home_by_each_hour|part_time_work_behavior_devices|full_time_work_behavior_devices|    destination_cbgs|delivery_behavior_devices|median_non_home_dwell_time|candidate_device_count|bucketed_away_from_home_time|median_percentage_tim

In [ ]:
# View schema
df_soc.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- cbg: string (nullable = true)
 |-- date_range_start: string (nullable = true)
 |-- date_range_end: string (nullable = true)
 |-- device_count: string (nullable = true)
 |-- distance_traveled_from_home: string (nullable = true)
 |-- bucketed_distance_traveled: string (nullable = true)
 |-- median_dwell_at_bucketed_distance_traveled: string (nullable = true)
 |-- completely_home_device_count: string (nullable = true)
 |-- median_home_dwell_time: string (nullable = true)
 |-- bucketed_home_dwell_time: string (nullable = true)
 |-- at_home_by_each_hour: string (nullable = true)
 |-- part_time_work_behavior_devices: string (nullable = true)
 |-- full_time_work_behavior_devices: string (nullable = true)
 |-- destination_cbgs: string (nullable = true)
 |-- delivery_behavior_devices: string (nullable = true)
 |-- median_non_home_dwell_time: string (nullable = true)
 |-- candidate_device_count: string (nullable = true)
 |-- bucketed_away_from_home_ti

In [ ]:
# Take small sample of data to experiment with
# df_soc = df_soc.limit(100)
# df_soc.show()

time: 7.01 ms (started: 2021-04-06 17:22:46 +00:00)


In [ ]:
df_soc.createOrReplaceTempView('clean_la')
df_soc = spark.sql('SELECT cbg, date_range_start, device_count, completely_home_device_count, part_time_work_behavior_devices, full_time_work_behavior_devices FROM clean_la')
df_soc.show()

+------------+--------------------+------------+----------------------------+-------------------------------+-------------------------------+
|         cbg|    date_range_start|device_count|completely_home_device_count|part_time_work_behavior_devices|full_time_work_behavior_devices|
+------------+--------------------+------------+----------------------------+-------------------------------+-------------------------------+
|060377021023|2020-02-07T00:00:...|          10|                           4|                              2|                              1|
|060372371013|2020-03-06T00:00:...|          10|                           3|                              2|                              1|
|060375334032|2020-02-14T00:00:...|          10|                           5|                              1|                              1|
|060372736001|2020-02-07T00:00:...|          10|                           1|                              1|                              2|
|06037

In [ ]:
df_soc = spark.sql('SELECT * FROM clean_la cl WHERE cl.device_count > 50')
df_soc.show()

+----+------------+--------------------+--------------------+------------+---------------------------+--------------------------+------------------------------------------+----------------------------+----------------------+------------------------+--------------------+-------------------------------+-------------------------------+--------------------+-------------------------+--------------------------+----------------------+----------------------------+---------------------------+-----------------------------+
| _c0|         cbg|    date_range_start|      date_range_end|device_count|distance_traveled_from_home|bucketed_distance_traveled|median_dwell_at_bucketed_distance_traveled|completely_home_device_count|median_home_dwell_time|bucketed_home_dwell_time|at_home_by_each_hour|part_time_work_behavior_devices|full_time_work_behavior_devices|    destination_cbgs|delivery_behavior_devices|median_non_home_dwell_time|candidate_device_count|bucketed_away_from_home_time|median_percentage_tim

In [ ]:
# View schema
df_soc.printSchema()

root
 |-- cbg: string (nullable = true)
 |-- date_range_start: string (nullable = true)
 |-- device_count: string (nullable = true)
 |-- completely_home_device_count: string (nullable = true)
 |-- part_time_work_behavior_devices: string (nullable = true)
 |-- full_time_work_behavior_devices: string (nullable = true)

time: 9.97 ms (started: 2021-04-06 17:22:46 +00:00)


In [ ]:
# Add completely home percentage column
df_soc = df_soc.withColumn('completely_home_percentage', (df_soc['completely_home_device_count']/df_soc['device_count']) * 100)
df_soc = df_soc.withColumn('part_time_work_percentage', (df_soc['part_time_work_behavior_devices']/df_soc['device_count']) * 100)
df_soc = df_soc.withColumn('full_time_work_percentage', (df_soc['full_time_work_behavior_devices']/df_soc['device_count']) * 100)

df_soc.show()

+----+------------+--------------------+--------------------+------------+---------------------------+--------------------------+------------------------------------------+----------------------------+----------------------+------------------------+--------------------+-------------------------------+-------------------------------+--------------------+-------------------------+--------------------------+----------------------+----------------------------+---------------------------+-----------------------------+--------------------------+-------------------------+-------------------------+
| _c0|         cbg|    date_range_start|      date_range_end|device_count|distance_traveled_from_home|bucketed_distance_traveled|median_dwell_at_bucketed_distance_traveled|completely_home_device_count|median_home_dwell_time|bucketed_home_dwell_time|at_home_by_each_hour|part_time_work_behavior_devices|full_time_work_behavior_devices|    destination_cbgs|delivery_behavior_devices|median_non_home_dwell

In [ ]:
df_soc.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- cbg: string (nullable = true)
 |-- date_range_start: string (nullable = true)
 |-- date_range_end: string (nullable = true)
 |-- device_count: string (nullable = true)
 |-- distance_traveled_from_home: string (nullable = true)
 |-- bucketed_distance_traveled: string (nullable = true)
 |-- median_dwell_at_bucketed_distance_traveled: string (nullable = true)
 |-- completely_home_device_count: string (nullable = true)
 |-- median_home_dwell_time: string (nullable = true)
 |-- bucketed_home_dwell_time: string (nullable = true)
 |-- at_home_by_each_hour: string (nullable = true)
 |-- part_time_work_behavior_devices: string (nullable = true)
 |-- full_time_work_behavior_devices: string (nullable = true)
 |-- destination_cbgs: string (nullable = true)
 |-- delivery_behavior_devices: string (nullable = true)
 |-- median_non_home_dwell_time: string (nullable = true)
 |-- candidate_device_count: string (nullable = true)
 |-- bucketed_away_from_home_ti

In [ ]:
# create temp view
df_soc.createOrReplaceTempView('new_clean_la')


time: 28.2 ms (started: 2021-04-10 00:31:49 +00:00)


In [ ]:
# get mean percentage for each cbg
df_soc_ch = spark.sql('SELECT cbg, AVG(completely_home_percentage) AS mean_completely_home_percentage FROM new_clean_la GROUP BY cbg')
df_soc_ch.createOrReplaceTempView('mean_completely_home')
df_soc_ch.show()

+------------+-------------------------------+
|         cbg|mean_completely_home_percentage|
+------------+-------------------------------+
|060590992242|                37.420104639405|
|060371278032|              38.92485225492883|
|060374808042|             38.001190909678655|
|060371042031|               32.8035338908155|
|060372123031|              37.22036369480373|
|060374015001|              33.03089182902592|
|060374087052|             39.495288482076084|
|060591100032|              32.95606850138264|
|060590744051|              26.19888833252573|
|060371910003|              35.29206186257467|
|060590994131|              32.10958775729351|
|060375743006|             23.355491502387743|
|060372145021|               38.3183930321702|
|060372948101|             26.935250797625933|
|060591101182|              31.76213793915933|
|060371390011|             31.862878639353053|
|060372243202|              31.51137648041673|
|060590525061|              29.21415418048064|
|060374328022

TEST FOR JUST COMPLETELY HOME

In [ ]:
df_mobility = df_soc_ch

time: 4.39 ms (started: 2021-04-10 00:38:16 +00:00)


In [ ]:
# get mean percentage for full time work for each cbg
df_soc_ft = spark.sql('SELECT cbg, AVG(full_time_work_percentage) AS mean_full_time_work_percentage FROM new_clean_la GROUP BY cbg')
df_soc_ft.createOrReplaceTempView('mean_fulltime_work')
df_soc_ft.show()

+------------+------------------------------+
|         cbg|mean_full_time_work_percentage|
+------------+------------------------------+
|060371042031|             4.248111802476035|
|060374034051|            3.7644396145056676|
|060590758091|             5.105389315235292|
|060372123031|             4.397899870071157|
|060590744051|             6.344504018382388|
|060371958032|             9.075850955124793|
|060372145021|             6.013009380113766|
|060590525061|            3.7280949789630533|
|060376021032|             6.334062954980371|
|060372122022|             5.575690029874976|
|060375743006|             3.102557768910321|
|060372948101|            4.5388564748836036|
|060372198001|             6.360041541084496|
|060371244001|             4.921585510048442|
|060375720012|             5.326588955041943|
|060375701002|            3.8417850605285135|
|060375401014|             6.022240466887228|
|060375540023|             4.818083949553114|
|060373107021|             4.75903

In [ ]:
# get mean percentage for full time work for each cbg
df_soc_pt = spark.sql('SELECT cbg, AVG(part_time_work_percentage) AS mean_part_time_work_percentage FROM new_clean_la GROUP BY cbg')
df_soc_pt.createOrReplaceTempView('mean_parttime_work')
df_soc_pt.show()

+------------+------------------------------+
|         cbg|mean_part_time_work_percentage|
+------------+------------------------------+
|060371042031|             6.562678906333603|
|060374034051|            5.2867894581488795|
|060590758091|             7.945659973111897|
|060372123031|             8.193909389381526|
|060590744051|             9.478888739065876|
|060371958032|             9.880581501214987|
|060372145021|             7.012580439461962|
|060590525061|             8.849011108830657|
|060376021032|             6.965781353948934|
|060372122022|             6.477831675918013|
|060375743006|             7.711839897859456|
|060372948101|             6.449241698043536|
|060372198001|             8.931200560647122|
|060371244001|             7.945335621850755|
|060375720012|             7.974893651032725|
|060375701002|             8.057050347734942|
|060375401014|             9.349470759861559|
|060375540023|             8.823036962752143|
|060373107021|             6.37593

In [ ]:
df_mobility = spark.sql('SELECT mean_completely_home.cbg, mean_completely_home.mean_completely_home_percentage, mean_fulltime_work.mean_full_time_work_percentage , mean_parttime_work.mean_part_time_work_percentage  FROM mean_completely_home INNER JOIN mean_fulltime_work ON mean_completely_home.cbg = mean_fulltime_work.cbg INNER JOIN mean_parttime_work ON mean_completely_home.cbg = mean_parttime_work.cbg')

time: 185 ms (started: 2021-04-06 17:28:37 +00:00)


In [ ]:
#df_mobility.show()
df_mobility.createOrReplaceTempView('mobility')


time: 16.7 ms (started: 2021-04-10 00:38:20 +00:00)




-----------------------
---





In [ ]:
# Read poverty data
# Read data into RDD
df_demographic = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/'la.csv'))
df_demographic.createOrReplaceTempView('demographic')
df_demographic.show()

+------------+---------+------------------+------------------+
|         cbg|pop_total|poverty_percentage|    perc_whiteonly|
+------------+---------+------------------+------------------+
|060373104003|     1286|17.884914463452567| 69.51788491446345|
|060590865023|     2545|22.789783889980352| 85.34381139489194|
|060376510024|     1285|               0.0|42.490272373540854|
|060376511011|     2708| 7.745398773006134| 50.70162481536189|
|060590994023|      672|12.969283276450511| 38.54166666666667|
|060590995094|      342| 17.83625730994152| 77.48538011695906|
|060376511012|     2483|2.6178010471204187|50.302053966975436|
|060376512011|      859|2.8846153846153846| 82.42142025611176|
|060376512012|     1179| 1.441899915182358|  74.1306191687871|
|060376512014|      989|2.4266936299292214| 70.57633973710819|
|060376512211|     3373|  9.95684340320592| 40.61666172546695|
|060590995092|      938| 5.756929637526652|  84.9680170575693|
|060590995095|      576|14.583333333333334|           7

In [ ]:
# join mobility and poverty
result = spark.sql('SELECT demographic.*, mobility.mean_completely_home_percentage, mobility.mean_full_time_work_percentage, mobility.mean_part_time_work_percentage FROM mobility INNER JOIN demographic ON mobility.cbg = demographic.cbg')
result.createOrReplaceTempView('demographic_mobility')
result.show()

+------------+---------+------------------+------------------+-------------------------------+------------------------------+------------------------------+
|         cbg|pop_total|poverty_percentage|    perc_whiteonly|mean_completely_home_percentage|mean_full_time_work_percentage|mean_part_time_work_percentage|
+------------+---------+------------------+------------------+-------------------------------+------------------------------+------------------------------+
|060371042031|     2577|  24.3892828999212| 57.00426852929763|               32.8148808521497|             4.248111802476035|             6.562678906333603|
|060371064071|     1997| 22.75515895147797| 73.41011517275913|             31.356802739418214|             6.511886123866137|             7.743105377455266|
|060371232031|     1748| 35.24027459954233|38.329519450800916|             32.717037953969005|             5.238361188397393|              7.60248095165861|
|060371244001|     1283|39.049103663289166| 85.42478565861

In [ ]:
# Count number of rows
spark.sql('SELECT COUNT(cbg) FROM demographic_mobility').show()

+----------+
|count(cbg)|
+----------+
|      8243|
+----------+

time: 1min 27s (started: 2021-04-06 17:33:07 +00:00)


In [ ]:
# sort by poverty
result = spark.sql('SELECT * FROM demographic_mobility ORDER BY CAST(poverty_percentage AS float) DESC')
result.createOrReplaceTempView('demographic_mobility')
result.show()

+------------+---------+------------------+------------------+-------------------------------+------------------------------+------------------------------+
|         cbg|pop_total|poverty_percentage|    perc_whiteonly|mean_completely_home_percentage|mean_full_time_work_percentage|mean_part_time_work_percentage|
+------------+---------+------------------+------------------+-------------------------------+------------------------------+------------------------------+
|060372227001|     4056| 88.46960167714884| 47.68244575936883|              20.07742122256767|             5.199924434188635|             7.995050842030239|
|060372247001|     2886| 87.87222523010287| 53.46500346500347|              30.32778196266256|            2.5674250773515634|             6.388250850219012|
|060372426002|     2289|  87.1559633027523|20.882481432940146|              30.31784431986003|             2.402896601401047|             4.666712928396625|
|060590745013|     1282|  86.1154446177847| 44.46177847113

In [ ]:
# group by poverty range


time: 2.46 ms (started: 2021-04-06 17:38:40 +00:00)


In [ ]:
result_df = result.toPandas()
result_df

,cbg,pop_total,poverty_percentage,perc_whiteonly,mean_completely_home_percentage,mean_full_time_work_percentage,mean_part_time_work_percentage
0,060372227001,4056,88.46960167714884,47.68244575936883,20.077421,5.199924,7.995051
1,060372247001,2886,87.87222523010287,53.46500346500347,30.327782,2.567425,6.388251
2,060372426002,2289,87.1559633027523,20.882481432940146,30.317844,2.402897,4.666713
3,060590745013,1282,86.1154446177847,44.46177847113885,34.458428,7.878765,9.162484
4,060372063002,2173,85.41862652869237,28.071790151863784,42.563031,3.329636,4.431804
...,...,...,...,...,...,...,...
8238,060371417003,489,0.0,95.0920245398773,25.338658,8.444944,9.167251
8239,060374802023,858,0.0,62.93706293706294,36.195509,6.305456,7.494102
8240,060376023022,571,0.0,52.71453590192644,28.423306,7.266807,10.162370
8241,060372771002,725,0.0,74.20689655172413,32.785335,6.786669,10.716680


time: 3min 10s (started: 2021-04-06 17:38:40 +00:00)


In [ ]:
# plot bar graph
alt.Chart(result_df).mark_point().encode(
    alt.X('poverty_percentage:Q'),
    alt.Y('mean_completely_home_percentage:Q'),
    tooltip=['cbg','poverty_percentage','mean_completely_home_percentage']
).properties(width=400, height=400).interactive()

alt.Chart(...)

time: 431 ms (started: 2021-04-06 17:41:50 +00:00)


In [ ]:
result_mean_df = result_df.groupby(pd.cut(result_df['poverty_percentage'].astype(float), np.arange(0,110, 10))).mean().reset_index()

time: 63.6 ms (started: 2021-04-06 17:41:50 +00:00)


In [ ]:
result_mean_df['poverty_percentage'] = result_mean_df['poverty_percentage'].astype(str)

time: 3.35 ms (started: 2021-04-06 17:41:50 +00:00)


In [ ]:
result_mean_df['color'] = ['#45a0d1' if x>30 else '#ff4833' for x in result_mean_df['mean_completely_home_percentage']]
result_mean_df

,poverty_percentage,mean_completely_home_percentage,mean_full_time_work_percentage,mean_part_time_work_percentage,color
0,"(0, 10]",33.332056,4.887086,7.777030,#45a0d1
1,"(10, 20]",34.585032,4.959354,7.538453,#45a0d1
2,"(20, 30]",34.918295,4.993906,7.444613,#45a0d1
3,"(30, 40]",35.210405,5.047852,7.275607,#45a0d1
4,"(40, 50]",35.010863,5.047666,7.191934,#45a0d1
5,"(50, 60]",34.638087,5.283669,7.178452,#45a0d1
6,"(60, 70]",33.698850,4.923943,7.096241,#45a0d1
7,"(70, 80]",34.354756,4.613792,6.960111,#45a0d1
8,"(80, 90]",30.746520,4.088038,6.518499,#45a0d1
9,"(90, 100]",NaN,NaN,NaN,#ff4833


time: 23.1 ms (started: 2021-04-06 17:46:30 +00:00)


In [ ]:
count = result_mean_df.shape[0]
print("{}".format(count))

10
time: 3.14 ms (started: 2021-04-06 17:47:25 +00:00)


In [ ]:
viz_df = []
for col in ['mean_completely_home_percentage','mean_full_time_work_percentage','mean_part_time_work_percentage']:
        for c in range(0,count):
            viz_df.append({'poverty': c, 'column': col, 'value': result_mean_df[col][c]})
viz_df = alt.Data(values=viz_df)

time: 13.1 ms (started: 2021-04-06 17:47:45 +00:00)


In [ ]:
#Visualizing data by mobility type

title = '{} {}'.format(city,'2020')
alt.Chart(viz_df, title=title).mark_line().encode(
    x=alt.X('poverty:O', title='poverty_percentage'),
    y=alt.Y('value:Q', type='quantitative', title='Percentage'),
    color='column:N'
).properties(
    width=300,
    height=250
).interactive()

alt.Chart(...)

time: 28.3 ms (started: 2021-04-06 17:47:56 +00:00)


In [ ]:
#
alt.Chart(result_mean_df).mark_point().encode(
    alt.X('poverty_percentage:N'),
    alt.Y('mean_completely_home_percentage'),
    color = alt.Color('color',scale=None),
    tooltip=[alt.Tooltip('poverty_percentage'),
             alt.Tooltip('mean_completely_home_percentage')]
).properties(width=400, height=400).interactive()

alt.Chart(...)

time: 30.9 ms (started: 2021-04-06 17:51:14 +00:00)


In [ ]:
result_df['poverty_percentage'].astype(float).dtype

dtype('float64')

time: 10.7 ms (started: 2021-04-06 17:51:40 +00:00)
